#### In this notebook we shall combine sentiment analysis score and product recommendation score , to predict the best suited products for users

In [1]:
import pandas as pd

- We saved the product recommendation results in user_final_rating, we shall use that

In [2]:
user_final_rating = pd.read_csv("user_final_rating.csv",index_col='reviews_username')

In [3]:
user_input = input("Enter your user name")
print(user_input)

Enter your user namejoshua
joshua


- We saved the df after cleaning and contatenating reviews title and text in df_clean , hence we shall use that

In [4]:
df_clean = pd.read_csv("df_clean.csv",index_col='Unnamed: 0')

df_clean

,reviews_title_text,name
0,just awesomei love this album its very good mo...,Pink Friday: Roman Reloaded Re-Up (w/dvd)
1,goodgood flavor this review was collected as p...,Lundberg Organic Cinnamon Toast Rice Cakes
2,goodgood flavor,Lundberg Organic Cinnamon Toast Rice Cakes
3,disappointedi read through the reviews on here...,K-Y Love Sensuality Pleasure Gel
4,irritationmy husband bought this gel for us th...,K-Y Love Sensuality Pleasure Gel
...,...,...
29995,softnessi got this conditioner with influenste...,L'or233al Paris Elvive Extraordinary Clay Reba...
29996,i love iti love it i received this for review...,L'or233al Paris Elvive Extraordinary Clay Reba...
29997,hair is so smooth after usefirst of all i love...,L'or233al Paris Elvive Extraordinary Clay Reba...
29998,perfect for my oily hairi received this throug...,L'or233al Paris Elvive Extraordinary Clay Reba...


Loading the log reg model and preparing X for prediction

In [5]:
#Loading the log reg model
import pickle
clf = pickle.load(open("XG_model_sm.pkl","rb"))
#Creating X and predicting y
X = df_clean['reviews_title_text'].values
from sklearn.feature_extraction.text import TfidfVectorizer
td = TfidfVectorizer(max_features = 4500)
X = td.fit_transform(X).toarray()
y =  clf.predict(X)
#Creating a column for predicted sentiment ie y
df_clean['predicted_sentiment'] = y

In [6]:
df_clean['predicted_sentiment'].value_counts()

1    24951
0     4796
Name: predicted_sentiment, dtype: int64

- Since now we have the sentiment analysis results, we will store it in df_predicted and export it for easy use when we deploy our app

In [7]:
df_predicted = df_clean
df_predicted.to_csv("df_predicted_xg.csv")

Creating a function to get top 5 recommended products

In [8]:
def get_top_5_sentiment_reommendation(user_input):
  d = user_final_rating.loc[user_input].sort_values(ascending=False)[0:20]
  dict_scores={}
  for a in d.index:
    #print(a,d[a])
    #Normalized Score for sentiment positive
    pos_score = df_clean[df_clean['name']==a].predicted_sentiment.value_counts(normalize=True)[1]
    rec_score = d[a]
    #Multiplying both
    final_score = pos_score*rec_score
   # print(a)
    dict_scores[a]=final_score
  b = sorted(dict_scores, key=dict_scores.get, reverse=True)[:5]
 # print(b)
  return b

In [9]:
get_top_5_sentiment_reommendation('joshua')

['Godzilla 3d Includes Digital Copy Ultraviolet 3d/2d Blu-Ray/dvd',
 'My Big Fat Greek Wedding 2 (blu-Ray + Dvd + Digital)',
 'Planes: Fire Rescue (2 Discs) (includes Digital Copy) (blu-Ray/dvd)',
 'Clorox Disinfecting Wipes Value Pack Scented 150 Ct Total',
 'The Resident Evil Collection 5 Discs (blu-Ray)']

- We will use above function and df_predicted for our deployed app